In [ ]:

#receipe recommendation system



import requests

# Function to get user input
def get_user_input():
    print("Welcome to the Recipe Recommendation System!")
    user_ingredients = input("Enter the ingredients you have (comma-separated): ").lower().split(",")
    user_ingredients = [ingredient.strip() for ingredient in user_ingredients]

    print("\nEnter your dietary restrictions dynamically (e.g., 'no chicken, no beef, no gluten'): ")
    diet_restrictions = input("Enter dietary restrictions: ").lower().split(",")
    diet_restrictions = [restriction.strip().replace("no ", "") for restriction in diet_restrictions]

    return user_ingredients, diet_restrictions

# Function to fetch recipes from TheMealDB API
def fetch_recipes(ingredients):
    base_url = "https://www.themealdb.com/api/json/v1/1/filter.php"
    recipes = []

    # Fetch recipes for each ingredient
    for ingredient in ingredients:
        params = {"i": ingredient}
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json().get("meals", [])
            if data:
                recipes.extend(data)
        else:
            print(f"Error fetching recipes for {ingredient}: {response.status_code}")

    return recipes

# Function to filter recipes based on dietary restrictions
def filter_recipes_by_diet(recipes, diet_restrictions):
    if not diet_restrictions:
        return recipes  # No restrictions, return all recipes

    filtered_recipes = []
    for recipe in recipes:
        recipe_details = fetch_recipe_details(recipe["idMeal"])
        if not recipe_details:
            continue

        # Gather all ingredients for the recipe
        ingredients = [
            recipe_details.get(f"strIngredient{i}") for i in range(1, 21)
            if recipe_details.get(f"strIngredient{i}")
        ]

        # Check if any restricted ingredient is present
        if not any(restriction in ingredients for restriction in diet_restrictions):
            filtered_recipes.append(recipe)

    return filtered_recipes

# Function to fetch detailed recipe information
def fetch_recipe_details(recipe_id):
    base_url = "https://www.themealdb.com/api/json/v1/1/lookup.php"
    params = {"i": recipe_id}
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json().get("meals", [])[0]
    else:
        return None

# Function to display recommended recipes
def display_recommendations(recommendations):
    if not recommendations:
        print("\nNo recipes match your ingredients and dietary preferences.")
        return

    print("\nWe found the following recipes for you:")
    for idx, recipe in enumerate(recommendations[:5], start=1):  # Limit to 5 recipes
        print(f"\n{idx}. {recipe['strMeal']}")
        print(f"Recipe Category: {recipe.get('strCategory', 'Not Available')}")
        print(f"Cuisine: {recipe.get('strArea', 'Not Available')}")

        # Fetch and display detailed instructions
        details = fetch_recipe_details(recipe["idMeal"])
        if details:
            print(f"Ingredients:")
            for i in range(1, 21):  # Ingredients and measures are indexed
                ingredient = details.get(f"strIngredient{i}")
                measure = details.get(f"strMeasure{i}")
                if ingredient and ingredient.strip():
                    print(f"- {ingredient}: {measure}")
            print(f"Procedure: {details['strInstructions']}")
            print(f"Recipe URL: {details['strSource'] or 'No URL available'}")
        else:
            print("Detailed instructions are not available.")

def main():
    user_ingredients, diet_restrictions = get_user_input()
    recommendations = fetch_recipes(user_ingredients)
    filtered_recommendations = filter_recipes_by_diet(recommendations, diet_restrictions)
    display_recommendations(filtered_recommendations)

if __name__ == "__main__":
    main()


Welcome to the Recipe Recommendation System!
Enter the ingredients you have (comma-separated): Chicken, rice

Enter your dietary restrictions dynamically (e.g., 'no chicken, no beef, no gluten'): 
Enter dietary restrictions: vegan

We found the following recipes for you:

1. Brown Stew Chicken
Recipe Category: Not Available
Cuisine: Not Available
Ingredients:
- Chicken: 1 whole
- Tomato: 1 chopped
- Onions: 2 chopped
- Garlic Clove: 2 chopped
- Red Pepper: 1 chopped
- Carrots: 1 chopped
- Lime: 1
- Thyme: 2 tsp
- Allspice: 1 tsp 
- Soy Sauce: 2 tbs
- Cornstarch: 2 tsp
- Coconut Milk: 2 cups 
- Vegetable Oil: 1 tbs
Procedure: Squeeze lime over chicken and rub well. Drain off excess lime juice.
Combine tomato, scallion, onion, garlic, pepper, thyme, pimento and soy sauce in a large bowl with the chicken pieces. Cover and marinate at least one hour.
Heat oil in a dutch pot or large saucepan. Shake off the seasonings as you remove each piece of chicken from the marinade. Reserve the marina